In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [160]:
df = pd.read_csv("../Resources/elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv", delimiter=";")
df.head()

,Melding ID,Meldingstidspunkt,Meldingsdato,Meldingsklokkeslett,Starttidspunkt,Startdato,Startklokkeslett,Startposisjon bredde,Startposisjon lengde,Hovedområde start (kode),...,Art - FDIR,Art - gruppe (kode),Art - gruppe,Rundvekt,Lengdegruppe (kode),Lengdegruppe,Bruttotonnasje 1969,Bruttotonnasje annen,Bredde,Fartøylengde
0,1497177,01.01.2018,01.01.2018,00:00,31.12.2017,31.12.2017,00:00,"-60,35","-46,133",NaN,...,Antarktisk krill,506.0,Antarktisk krill,706714.0,5.0,28 m og over,9432.0,NaN,"19,87","133,88"
1,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Hyse,202.0,Hyse,9594.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
2,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Torsk,201.0,Torsk,8510.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
3,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Blåkveite,301.0,Blåkveite,196.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
4,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Sei,203.0,Sei,134.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"


In [161]:
df.dtypes

Melding ID                    int64
Meldingstidspunkt            object
Meldingsdato                 object
Meldingsklokkeslett          object
Starttidspunkt               object
Startdato                    object
Startklokkeslett             object
Startposisjon bredde         object
Startposisjon lengde         object
Hovedområde start (kode)    float64
Hovedområde start            object
Lokasjon start (kode)       float64
Havdybde start                int64
Stopptidspunkt               object
Stoppdato                    object
Stoppklokkeslett             object
Varighet                      int64
Fangstår                      int64
Stopposisjon bredde          object
Stopposisjon lengde          object
Hovedområde stopp (kode)    float64
Hovedområde stopp            object
Lokasjon stopp (kode)       float64
Havdybde stopp                int64
Trekkavstand                float64
Redskap FAO (kode)           object
Redskap FAO                  object
Redskap FDIR (kode)         

In [162]:
df = df[['Starttidspunkt', 'Art - FDIR','Redskap FDIR', 'Hovedområde start', 'Varighet', 'Fartøylengde', 'Rundvekt']].copy()
df
# df = df[['Starttidspunkt', 'Art - FDIR','Redskap FDIR', 'Hovedområde start', 'Rundvekt']].copy()
# df

,Starttidspunkt,Art - FDIR,Redskap FDIR,Hovedområde start,Varighet,Fartøylengde,Rundvekt
0,31.12.2017,Antarktisk krill,Flytetrål,NaN,1440,"133,88",706714.0
1,30.12.2017 23:21,Hyse,Bunntrål,Bjørnøya,295,"56,8",9594.0
2,30.12.2017 23:21,Torsk,Bunntrål,Bjørnøya,295,"56,8",8510.0
3,30.12.2017 23:21,Blåkveite,Bunntrål,Bjørnøya,295,"56,8",196.0
4,30.12.2017 23:21,Sei,Bunntrål,Bjørnøya,295,"56,8",134.0
...,...,...,...,...,...,...,...
305429,31.12.2018 19:41,Gråsteinbit,Bunntrål,Vest-Spitsbergen,232,"57,3",145.0
305430,31.12.2018 19:41,Uer (vanlig),Bunntrål,Vest-Spitsbergen,232,"57,3",136.0
305431,31.12.2018 19:41,Flekksteinbit,Bunntrål,Vest-Spitsbergen,232,"57,3",132.0
305432,31.12.2018 19:41,Snabeluer,Bunntrål,Vest-Spitsbergen,232,"57,3",102.0


In [163]:
df = df.dropna()
df

,Starttidspunkt,Art - FDIR,Redskap FDIR,Hovedområde start,Varighet,Fartøylengde,Rundvekt
1,30.12.2017 23:21,Hyse,Bunntrål,Bjørnøya,295,"56,8",9594.0
2,30.12.2017 23:21,Torsk,Bunntrål,Bjørnøya,295,"56,8",8510.0
3,30.12.2017 23:21,Blåkveite,Bunntrål,Bjørnøya,295,"56,8",196.0
4,30.12.2017 23:21,Sei,Bunntrål,Bjørnøya,295,"56,8",134.0
5,31.12.2017 05:48,Hyse,Bunntrål,Bjørnøya,267,"56,8",9118.0
...,...,...,...,...,...,...,...
305429,31.12.2018 19:41,Gråsteinbit,Bunntrål,Vest-Spitsbergen,232,"57,3",145.0
305430,31.12.2018 19:41,Uer (vanlig),Bunntrål,Vest-Spitsbergen,232,"57,3",136.0
305431,31.12.2018 19:41,Flekksteinbit,Bunntrål,Vest-Spitsbergen,232,"57,3",132.0
305432,31.12.2018 19:41,Snabeluer,Bunntrål,Vest-Spitsbergen,232,"57,3",102.0


In [164]:
df = df.copy()

df['Starttidspunkt'] = pd.to_datetime(df['Starttidspunkt'], format='mixed')
df['month'] = df['Starttidspunkt'].dt.month
df['day'] = df['Starttidspunkt'].dt.day
df['hour'] = df['Starttidspunkt'].dt.hour
df = df.drop(['Starttidspunkt'], axis=1)

df

,Art - FDIR,Redskap FDIR,Hovedområde start,Varighet,Fartøylengde,Rundvekt,month,day,hour
1,Hyse,Bunntrål,Bjørnøya,295,"56,8",9594.0,12,30,23
2,Torsk,Bunntrål,Bjørnøya,295,"56,8",8510.0,12,30,23
3,Blåkveite,Bunntrål,Bjørnøya,295,"56,8",196.0,12,30,23
4,Sei,Bunntrål,Bjørnøya,295,"56,8",134.0,12,30,23
5,Hyse,Bunntrål,Bjørnøya,267,"56,8",9118.0,12,31,5
...,...,...,...,...,...,...,...,...,...
305429,Gråsteinbit,Bunntrål,Vest-Spitsbergen,232,"57,3",145.0,12,31,19
305430,Uer (vanlig),Bunntrål,Vest-Spitsbergen,232,"57,3",136.0,12,31,19
305431,Flekksteinbit,Bunntrål,Vest-Spitsbergen,232,"57,3",132.0,12,31,19
305432,Snabeluer,Bunntrål,Vest-Spitsbergen,232,"57,3",102.0,12,31,19


In [165]:
df = pd.get_dummies(df, columns=['Hovedområde start'])
df = pd.get_dummies(df, columns=['Redskap FDIR'])
df = pd.get_dummies(df, columns=['Art - FDIR'])
df

,Varighet,Fartøylengde,Rundvekt,month,day,hour,Hovedområde start_Admiralityfeltet,Hovedområde start_Bjørnøya,Hovedområde start_Britvinfeltet,Hovedområde start_Danmarkstredet,...,Art - FDIR_Uer (vanlig),"Art - FDIR_Uer, uspes.",Art - FDIR_Vanlig fløyfisk,"Art - FDIR_Vanlig ti-armet blekksprut, uspes.",Art - FDIR_Vanlig ulke,Art - FDIR_Vassild,Art - FDIR_Villsvinfisk,Art - FDIR_Vågehval,Art - FDIR_Ål,Art - FDIR_Øyepål
1,295,"56,8",9594.0,12,30,23,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,295,"56,8",8510.0,12,30,23,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,295,"56,8",196.0,12,30,23,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,295,"56,8",134.0,12,30,23,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
5,267,"56,8",9118.0,12,31,5,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305429,232,"57,3",145.0,12,31,19,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
305430,232,"57,3",136.0,12,31,19,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
305431,232,"57,3",132.0,12,31,19,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
305432,232,"57,3",102.0,12,31,19,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [166]:
df.dtypes

Varighet                     int64
Fartøylengde                object
Rundvekt                   float64
month                        int32
day                          int32
                            ...   
Art - FDIR_Vassild            bool
Art - FDIR_Villsvinfisk       bool
Art - FDIR_Vågehval           bool
Art - FDIR_Ål                 bool
Art - FDIR_Øyepål             bool
Length: 195, dtype: object

In [167]:
df['Fartøylengde']=df['Fartøylengde'].str.replace(',', '.').astype(float)

In [168]:
# df = df[['Rundvekt', 'Varighet', 'month', 'day', 'hour', 'Fartøylengde']]
# df = df.corr().round(2)
# df


In [169]:
def map_to_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['season'] = df['month'].apply(map_to_season)

df = pd.get_dummies(df, columns=['season'])
df = df.drop(['month'], axis=1)

In [170]:
X = df.drop(['Rundvekt'], axis=1)
y = df[['Rundvekt']]



In [171]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y_train.dtypes

Rundvekt    float64
dtype: object

In [172]:
knn_pipe = Pipeline([('mms', MinMaxScaler()),
                     ('knn', KNeighborsRegressor())])

In [173]:
params = [{'knn__n_neighbors': [3, 5, 7, 9],
         'knn__weights': ['uniform', 'distance'],
         'knn__leaf_size': [15, 20]}]

In [174]:
gs_knn = GridSearchCV(knn_pipe,
                      param_grid=params,
                      scoring='neg_mean_squared_error',
                      cv=5)
gs_knn.fit(X_train, y_train)
gs_knn.best_params_

{'knn__leaf_size': 15, 'knn__n_neighbors': 9, 'knn__weights': 'uniform'}

In [175]:
predictions = gs_knn.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print("MSE: {:.2f}".format(mse))
print("r2 score: {:.2f}".format(r2))

MSE: 417231601.47
r2 score: 0.69
